In [30]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

In [15]:
data1 = pd.read_csv(path_raw + 'clean_comentarios_facebook.csv', sep=',', engine='python')
data2 = pd.read_csv(path_raw + 'labeled_corpus_6K.txt', sep=';\\|\\|;', engine='python', names=['id','twt', 'label'])
data3 = pd.read_csv(path_raw + 'sp_tweets.csv', sep=',', engine='python')
data4 = pd.read_csv(path_raw + 'train_dev_es_merged.tsv', sep='\t')
data5 = pd.read_csv(path_raw +'hascosva_2022_anonymized.tsv', sep='\t')

#hateval2019
data6 = pd.read_csv(path_raw +'hateval2019_es_train.csv', sep=',', engine='python')
data7 = pd.read_csv(path_raw +'hateval2019_es_test.csv', sep=',', engine='python')
data8 = pd.read_csv(path_raw +'hateval2019_es_dev.csv', sep=',', engine='python')

data9 = pd.read_csv(path_raw +'corpusTradLabeled.csv', sep=',', engine='python')
data10 = pd.read_csv(path_raw +'jigsaw-toxic-comment-train-google-es-cleaned.csv', sep=',', engine='python')

In [47]:
toxicData = []
normalData = []

normalData.extend(data1.loc[data1['Category'] == 0, 'Text'].tolist())
toxicData.extend(data1.loc[data1['Category'] == 1, 'Text'].tolist())

normalData.extend(data2.loc[data2['label'] == 0, 'twt'].tolist())
toxicData.extend(data2.loc[data2['label'] == 1, 'twt'].tolist())

toxicData.extend(data3['tweet'].tolist())

normalData.extend(data4.loc[data4['HS'] == 0, 'text'].tolist())
toxicData.extend(data4.loc[data4['HS'] == 1, 'text'].tolist())

normalData.extend(data5.loc[data5['label'] == 0, 'text'].tolist())
toxicData.extend(data5.loc[data5['label'] == 1, 'text'].tolist())

normalData.extend(data6.loc[data6['HS'] == 0, 'text'].tolist())
normalData.extend(data7.loc[data7['HS'] == 0, 'text'].tolist())
normalData.extend(data8.loc[data8['HS'] == 0, 'text'].tolist())
toxicData.extend(data6.loc[data6['HS'] == 1, 'text'].tolist())
toxicData.extend(data7.loc[data7['HS'] == 1, 'text'].tolist())
toxicData.extend(data8.loc[data8['HS'] == 1, 'text'].tolist())

normalData.extend(data9.loc[data9['label2'] == 0, 'text'].tolist())
toxicData.extend(data9.loc[data9['label2'] == 1, 'text'].tolist())

normalData.extend(data10.loc[data10['toxic'] == 0, 'comment_text'].tolist())
toxicData.extend(data10.loc[data10['toxic'] == 1, 'comment_text'].tolist())

In [48]:
toxic = pd.DataFrame({'twt': toxicData, 'label': 1})
normal = pd.DataFrame({'twt': normalData, 'label': 0})

In [56]:
df =  pd.concat([toxic, normal], ignore_index=True)
df["label"] =  df["label"].astype(bool)
df["twt"] =  df["twt"].astype(str)

In [57]:
import re
import preprocessor as p
import string
from tqdm import tqdm
def clean_text(text):
    # Clean with tweet preprocessor: remove URLs, Hashtags, Mentions, Emojis, Smileys
    text = p.clean(text) 

    # Convert text to lowercase, remove punctuation, characters with numbers, and newlines
    text = text.lower()
    text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub(r'\n', '', text)
    text = re.sub(r'[0-9]+', '', text)
    text = re.sub(r'(\w)\1{2,}', r'\1\1', text)
    text = text.strip()

    return text

In [58]:
df['twt_cleaned'] = df['twt'].apply(clean_text)

In [59]:
df.replace('', np.nan, inplace=True)
df = df.dropna()
df.reset_index(inplace=True)
df

,index,twt,label,twt_cleaned
0,0,y pensar ganar respetar ver aborrencimiento de...,True,y pensar ganar respetar ver aborrencimiento de...
1,1,a a holgar ver hombre necesitar coser obvio ne...,True,a a holgar ver hombre necesitar coser obvio ne...
2,2,drogar mocoso q empezar a hablar reportaje met...,True,drogar mocoso q empezar a hablar reportaje met...
3,3,mujer feminazi destruir año verdadero luchar f...,True,mujer feminazi destruir ao verdadero luchar fe...
4,4,feminista feminoide,True,feminista feminoide
...,...,...,...,...
444644,447146,"""\n\n == Homosexualidad - Discrepancia ==\n\n ...",False,homosexualidad discrepancia este artculo esta...
444645,447147,:: ¿Consenso para arruinar Wikipedia? Creo que...,False,consenso para arruinar wikipedia creo que eso ...
444646,447148,Acabo de descubrir otra lista: la lista de com...,False,acabo de descubrir otra lista la lista de comp...
444647,447149,"""\n He estado leyendo esta página de discusión...",False,he estado leyendo esta pgina de discusin y me ...


In [60]:
def split_and_save(data, prefix, test_size=0.2, dev_size=0.1, random_state=42):
    
    train_data, temp_data = train_test_split(data, test_size=(test_size + dev_size), random_state=random_state)
    
    test_data, dev_data = train_test_split(temp_data, test_size=(dev_size / (test_size + dev_size)), random_state=random_state)
    
    train_data.to_csv(f'./processed/train_{prefix}', index=False,header=False)
    test_data.to_csv(f'./processed/test_{prefix}', index=False,header=False)
    dev_data.to_csv(f'./processed/dev_{prefix}', index=False,header=False)

In [61]:
split_and_save(df[df['label']==1].twt_cleaned, 'toxic')
split_and_save(df[df['label']==0].twt_cleaned, 'normal')